In [1]:
import os
import sys
sys.path.insert(0, "/home/axiwa/Learn/CG/covariance/mipmap/build/python")
import mitsuba as mi
from matplotlib import pyplot as plt
mi.set_variant('cuda_ad_rgb')

In [2]:
# Sampler
total = 1
sampler = mi.load_dict({'type': 'independent', 'sample_count': total})
sampler.seed(seed = 2, wavefront_size = total)

In [3]:
# Scene
scene_path = "mipmap.xml"
scene = mi.load_file(scene_path)
image = mi.render(scene)
mi.Bitmap(image)

Bitmap[
  pixel_format = rgb,
  component_format = float32,
  size = [1024, 1024],
  srgb_gamma = 0,
  struct = Struct<12>[
    float32 R; // @0, premultiplied alpha
    float32 G; // @4, premultiplied alpha
    float32 B; // @8, premultiplied alpha
  ],
  data = [ 12 MiB of image data ]
]

In [4]:
sensor = scene.sensors()[0]
viewpoint = sensor.world_transform() @ mi.ScalarPoint3f(0)
to_world = sensor.world_transform()
print (viewpoint)
print (to_world)

[[-19.134172439575195, 2.0, -46.19397735595703]]
[[[0.92388, 0, 0.382683, -19.1342],
  [0, 1, 0, 2],
  [-0.382683, 0, 0.92388, -46.194],
  [0, 0, 0, 1]]]


In [5]:
ray = mi.RayDifferential3f(viewpoint, mi.Vector3f([48 / 4096, -2, 25 + 48 / 4096]))
print (ray)
wo = mi.RayDifferential3f([0, 0, 0], mi.Vector3f([0, 1, 0]))

Ray3fDC[
  o = [[-19.1342, 2, -46.194]],
  d = [[0.0117188, -2, 25.0117]],
  maxt = [3.40282e+38],
  time = [0],
]


In [6]:
si = scene.ray_intersect(ray)
si.duv_dx = [0.00001, 0.00001]
si.duv_dy = [0.00001, 1/1024]
print (si)

SurfaceInteraction[
  t = [1],
  time = [0],
  wavelengths = [],
  p = [[-19.1225, 0, -21.1823]],
  shape = [0x7ff2380130d0],
  uv = [[0.117551, 0.569413]],
  n = [[0, 1, 0]],
  sh_frame = Frame[
    s = [[1, -0, -0]],
    t = [[0, 0, -1]],
    n = [[0, 1, 0]]
  ],
  dp_du = [[50, -0, -0]],
  dp_dv = [[-0, -0, 55]],
  dn_du = [[0, 0, 0]],
  dn_dv = [[0, 0, 0]],
  duv_dx = [[1e-05, 1e-05]],
  duv_dy = [[1e-05, 0.000976562]],
  wi = [[-0.0117188, 25.0117, 2]],
  prim_index = [0],
  instance = [0x0]
]


In [7]:
ctx = mi.BSDFContext()

bsdf = mi.load_dict({      
    'type': 'diffuse',
    'reflectance': {
        'type': 'bitmap2',
        'filename': 'figures/check16.png',
        'filter_type': 'nearest',
        'wrap_mode': 'repeat',
        'mipmap_filter_type': 'trilinear',
        'to_uv': mi.ScalarTransform4f.scale([1, 1, 1])
    }
})

2023-03-13 16:10:08 WARN main [BitmapTexture2] Mipmap filter may not be compatible with texture filter

MIPMAP BUILT SUCCESS


In [8]:
bs, spec = bsdf.sample(ctx, si, sampler.next_1d(), sampler.next_2d())
print (bs)
print (spec)

BSDFSample[
  wo = [[0.304661, 0.653802, 0.692621]],
  pdf = [0.220468],
  eta = [1],
  sampled_type = TODO,
  sampled_component = [0]
]
[[0.0, 0.0, 0.0]]
